In [49]:
#Package imported

import time
import pandas as pd
import datetime
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [57]:
#Function for extracting the required data in the fin-streamer section in the lxml

def content (web_content, stock_index,data_field):
    try:
        if data_field == "regularMarketVolume":
            texts=float((web_content.find_all('fin-streamer',{'data-symbol': "AAPL","data-field":data_field})[0].text).replace(",",""))
        else:
            texts=float((web_content.find_all('fin-streamer',{'data-symbol': "AAPL","data-field":data_field})[0].text))
    except IndexError:
        texts = []

    return texts




#Funcion for extracting the information from the lxml in the yahoo finance.

def real_time_stock_data(stock_code):
    data_fields=["regularMarketPrice","regularMarketVolume","trailingPE"]
    texts=[]
    url = 'https://finance.yahoo.com/quote/' +stock_code +'/'
    try:
        r = requests.get(url)
        web_content = BeautifulSoup(r.text, 'lxml')
        for data_field in data_fields:
            texts.append(content(web_content, stock_code,data_field))
        if texts != []:
            result = texts
        else:
            result = []
    except ConnectionError:
         result = []

    return result


In [53]:
#Initialize the KafkaProdcuer

producer =KafkaProducer(bootstrap_servers=['ed-kafka:29092'],
                        api_version=(3,7,1), 
                        value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [ ]:
#webscraping the data and pass it to the kafka topic

while True:
    data= real_time_stock_data('AAPL')
    producer.send('streaming', value={"Price":data[0],"Vloume":data[1],"TrailingPE":data[2]})
    time.sleep(10)